In [1]:
# !pip install swig

In [2]:
# !pip install smac

In [3]:
# !pip install ConfigSpace

In [4]:
from ConfigSpace import Configuration, ConfigurationSpace

import numpy as np
from smac import HyperparameterOptimizationFacade, Scenario
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

iris = datasets.load_iris()


def train(config: Configuration, seed: int = 0) -> float:
    classifier = SVC(C=config["C"], random_state=seed)
    scores = cross_val_score(classifier, iris.data, iris.target, cv=5)
#     print (np.mean(scores))
    return 1 - np.mean(scores)


configspace = ConfigurationSpace({"C": (0.100, 1000.0)})

# Scenario object specifying the optimization environment
scenario = Scenario(configspace, deterministic=True, n_trials=200)

# Use SMAC to find the best configuration/hyperparameters
smac = HyperparameterOptimizationFacade(scenario, train)
incumbent = smac.optimize()

[INFO][abstract_initial_design.py:147] Using 10 initial design configurations and 0 additional configurations.
[INFO][smbo.py:497] Continuing from previous run.
[INFO][smbo.py:270] Optimization process was already finished. Returning incumbent...


In [5]:
incumbent.values


<bound method Mapping.values of Configuration(values={
  'C': 7.082646102284318,
})>

In [6]:
classifier = SVC(C=6.724148702862232)
scores = cross_val_score(classifier, iris.data, iris.target, cv=5)
print(np.mean(scores))

0.9866666666666667


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, ShuffleSplit
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
# from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
# import seaborn as sn
import warnings

warnings.filterwarnings('ignore')

In [8]:
# Load the Credit Card Fraud Detection dataset
url = "https://www.openml.org/data/get_csv/31/dataset_31.csv"
df = pd.read_csv(url)
df = df.dropna()
df = pd.get_dummies(df, drop_first=True)

In [9]:
df.head()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents,checking_status_'<0',checking_status_'>=200',checking_status_'no checking',...,other_payment_plans_none,other_payment_plans_stores,housing_own,housing_rent,job_'unemp/unskilled non res',job_'unskilled resident',job_skilled,own_telephone_yes,foreign_worker_yes,class_good
0,6,1169,4,4,67,2,1,True,False,False,...,True,False,True,False,False,False,True,True,True,True
1,48,5951,2,2,22,1,1,False,False,False,...,True,False,True,False,False,False,True,False,True,False
2,12,2096,2,3,49,1,2,False,False,True,...,True,False,True,False,False,True,False,False,True,True
3,42,7882,2,4,45,1,2,True,False,False,...,True,False,False,False,False,False,True,False,True,True
4,24,4870,3,4,53,2,2,True,False,False,...,True,False,False,False,False,False,True,False,True,False


In [10]:
df['class_good'].value_counts()

class_good
True     700
False    300
Name: count, dtype: int64

In [12]:
from sklearn.feature_selection import SelectKBest, f_classif

def Which_features(X_train,y_train,number_of_columns):

    # Select the top k features based on ANOVA F-statistic
    selector = SelectKBest(score_func=f_classif, k=number_of_columns)
    X_train_selected = selector.fit_transform(X_train, y_train)

    # Get the column names of the selected features
    selected_feature_names = X_train.columns[selector.get_support()]

    
    return list(selected_feature_names)

selected_features = Which_features(X_train_raw,y_train,number_of_columns=15)
selected_features

['duration',
 'credit_amount',
 'age',
 "checking_status_'<0'",
 "checking_status_'no checking'",
 "credit_history_'critical/other existing credit'",
 "credit_history_'no credits/all paid'",
 "purpose_'new car'",
 'purpose_radio/tv',
 "savings_status_'<100'",
 "property_magnitude_'no known property'",
 "property_magnitude_'real estate'",
 'other_payment_plans_none',
 'housing_own',
 'housing_rent']

In [14]:
X_train_raw.shape

(800, 15)

In [15]:
classifer=KNeighborsClassifier(n_neighbors=7)
classifer.fit(X_train_raw,y_train)
y_pred = classifer.predict(X_test)
loss=1-accuracy_score(y_test,y_pred)
print(loss)


0.32999999999999996


In [16]:
def plot_from_smac(smac) -> None:
    plt.figure()
    configs = smac.runhistory.get_configs()
    incumbents = smac.intensifier.get_incumbents()

    for i, config in enumerate(configs):
        if config in incumbents:
            continue

        label = None
        if i == 0:
            label = "Configuration"

        x = config["x"]
        f1, f2 = mymodell(x)
        plt.scatter(f1, f2, c="blue", alpha=0.1, marker="o", zorder=3000, label=label)

    for i, config in enumerate(incumbents):
        label = None
        if i == 0:
            label = "Incumbent"

        x = config["x"]
        f1, f2 = mymodell(x)
        plt.scatter(f1, f2, c="red", alpha=1, marker="x", zorder=3000, label=label)

    plt.xlabel("f1")
    plt.ylabel("f2")
    plt.title("Schaffer 2D")
    plt.legend()

    plt.show()

In [17]:
# !pip install statsmodels

In [18]:
from statsmodels.tsa.arima.model import ARIMA
def Arima(train,test, p, d, q,freq):
    train = train.asfreq(freq)
    test = test.asfreq(freq)
    
    
    model_fit = ARIMA(train['y'], order=(p, d, q))
    model_fit = model_fit.fit()
    forecast_values = model_fit.forecast(steps=len(test))
        

    return  np.mean(np.abs((forecast_values - test['y']) / test['y'])) * 100




ModuleNotFoundError: No module named 'statsmodels'

In [82]:
from ConfigSpace import Categorical, Configuration, ConfigurationSpace, Float, Integer
from ConfigSpace.conditions import InCondition,EqualsCondition
from smac import HyperparameterOptimizationFacade, Scenario
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRegressor
from smac import Callback
import time
#potential optimizations
#add timeserie
#search for the best hypers parameters for the each model
#add random state parameters
#what is the problem with linear svc
class CustomCallback(Callback):
    def __init__(self, max_same_incumbent_trials=30):
        self.trials_counter = 0
        self.incumbent_counter = 0
        self.last_incumbent = None
        self.max_same_incumbent_trials = max_same_incumbent_trials

    def on_start(self, smbo):
        print("let's start the optimization")

    def on_tell_end(self, smbo, info, value):
        self.trials_counter += 1
        print(f"The number of trials is: {self.trials_counter}")

        incumbents = smbo.intensifier.get_incumbents()
        if incumbents:
            current_incumbent = incumbents[0]  # Assuming there's at least one incumbent
            current_incumbent_dict = current_incumbent.get_dictionary()

            if self.last_incumbent == current_incumbent_dict:
                self.incumbent_counter += 1
            else:
                self.incumbent_counter = 0  # Reset the counter if incumbent changes
                self.last_incumbent = current_incumbent_dict

            if self.incumbent_counter >= self.max_same_incumbent_trials:
                print(f"The incumbent has remained the same for {self.max_same_incumbent_trials} trials.")
                print("Stopping the optimization process.")
                return False  # This stops the optimization process

            if self.trials_counter % 10 == 0:
                print(f"The incumbent is: {current_incumbent_dict}")
                print(f"The incumbent loss is: {smbo.runhistory.get_cost(current_incumbent)}")

            if self.trials_counter == 100:
                print("Let's stop the optimization at trial 100")
                return False  # This stops the optimization process after 100 trials

        return None

class Models:
    def __init__(self,similarModels,problemType,X_train,Y_train,X_test,Y_test,freq='d',m=7):
        self.Models=similarModels
        self.Problemtype=problemType
        self.X_train=X_train
        self.Y_train=Y_train
        self.X_test=X_test
        self.Y_test=Y_test
        self.freq=freq
        self.m=m
    def configspace(self):
        confs = ConfigurationSpace(seed=0)
        #HPOs
        if self.Problemtype=='Classification':

            # models are ['KNN','LR',"RF",'SVC']
            models=Categorical('Models',self.Models)
            parameters=[]
            conditions=[]
            parameters.append(models)
            if 'KNN' in self.Models:
                Kneighbors=Integer('Ks',(1,10),default=1)
                useks=InCondition(child=Kneighbors,parent=models,values=['KNN'])
                parameters.append(Kneighbors)
                conditions.append(useks)
            if 'SVC' in self.Models:
                kernel=Categorical('kernel',['linear','rbf'])
                usekernel=InCondition(child=kernel,parent=models,values=['SVC'])
                parameters.append(kernel)
                conditions.append(usekernel)
                if 'LR' in self.Models:
                    rc=Float('regularizationStre',(0.01,1))
                    userc=InCondition(child=rc,parent=models,values=['LR','SVC'])
                    parameters.append(rc)
                    conditions.append(userc)
                else:
                    rc=Float('regularizationStre',(0.01,1))
                    userc=InCondition(child=rc,parent=models,values=['SVC'])
                    parameters.append(rc)
                    conditions.append(userc)
            elif 'LR' in self.Models:
                rc=Float('regularizationStre',(0.01,1))
                userc=InCondition(child=rc,parent=models,values=['LR'])
                parameters.append(rc)
                conditions.append(userc)
            if 'RF' in self.Models:
                nestimators=Integer('n_estimators',(1,20),default=10)
                useEst=InCondition(child=nestimators,parent=models,values=['RF'])
                parameters.append(nestimators)
                conditions.append(useEst)
            # print("the parameters are: ",parameters)
            # print("the conditions are: ",conditions)
            confs.add_hyperparameters(parameters)
            confs.add_conditions(conditions)
        elif self.Problemtype=='Regression':
            models=Categorical('Models',self.Models)
            #linear regression parameters
            #lasso and ridge regression parameters
            
            #random forest and XGboost parameters
            # nestimators=Integer('n_estimators',(1,20),default=10)
            #dependencies 
            conditions=[]
            parameters=[models]
            if 'Lasso' in self.Models:
                alphalasso=Float('alphalas',(0.01,100))
                usealphaLASSO=InCondition(child=alphalasso,parent=models,values=['Lasso'])
                parameters.append(alphalasso)
                conditions.append(usealphaLASSO)
            if 'Ridge' in self.Models:
                alpharidge=Float('alpharid',(0.01,100))
                usealphaRidge=InCondition(child=alpharidge,parent=models,values=['Ridge'])
                parameters.append(alpharidge)
                conditions.append(usealphaRidge)
            if 'RF' in self.Models:
                nestimatorsrf=Integer('n_estimatorsrf',(1,20),default=10)
                useEstRf=InCondition(child=nestimatorsrf,parent=models,values=['RF'])
                parameters.append(nestimatorsrf)
                conditions.append(useEstRf)
            if 'XGboost' in self.Models:
                nestimators=Integer('n_estimatorsxg',(1,20),default=10)
                useEstXG=InCondition(child=nestimators,parent=models,values=['XGboost'])
                parameters.append(nestimators)
                conditions.append(useEstXG)
            confs.add_hyperparameters(parameters)
            confs.add_conditions(conditions)
        elif self.Problemtype=='TimeSeries':
            # ['arima',sarima,'prophet']
            models=Categorical("Models",self.Models)
            
            p=Integer('p',(0,3))
            q=Integer('q',(0,3))
            d=Integer('d',(0,3))

            conditions=[]
            parameters=[models,p,q,d]
            if 'Sarima' in self.Models:
                SarimaP=Integer('sp',(0,3))
                sarimaq=Integer('sq',(0,3))
                sarimad=Integer('sd',(0,3))
                
                usecondip=EqualsCondition(child=SarimaP,parent=models,value='Sarima')
                usecondiq=EqualsCondition(child=sarimaq,parent=models,value='Sarima')
                usecondid=EqualsCondition(child=sarimad,parent=models,value='Sarima')
                parameters.append(SarimaP)
                parameters.append(sarimaq)
                parameters.append(sarimad)
                conditions.append(usecondip)
                conditions.append(usecondiq)
                conditions.append(usecondid)

            confs.add_hyperparameters(parameters)
            confs.add_conditions(conditions)
            
        return confs
            
        
    def train(self,config:Configuration,seed: int=0):
        start_time=time.time()
        config_dict=config.get_dictionary()
        model=config_dict['Models']
        print(f"config_dict:{config_dict}")
        if self.Problemtype=='Classification':
            return self.classification(config_dict,start_time)
        elif self.Problemtype=='Regression':
            return self.regression(config_dict,start_time)
        elif self.Problemtype=='TimeSeries':
            return self.timeser(config_dict)
    def classification(self,configDict,start_time):
        model=configDict['Models']
        if model=='KNN':
            Classifier=KNeighborsClassifier(n_neighbors=configDict['Ks'])
        elif model=='LR':
            Classifier=LogisticRegression(C=configDict['regularizationStre'])
        elif model=='RF':
            Classifier=RandomForestClassifier(n_estimators=configDict['n_estimators'],random_state=42)
        elif model=='SVC':
            Classifier=SVC(C=configDict['regularizationStre'],kernel=configDict['kernel'])
        print(f"the type of the classifier is: {type(Classifier)}")
        Classifier.fit(self.X_train,self.Y_train)
        y_pred = Classifier.predict(self.X_test)
        loss=1-accuracy_score(self.Y_test,y_pred)
        print("the loss is: ",loss)
        return {'loss':loss,'time':time.time()-start_time}
    def regression(self, configDict,start_time):
            model=configDict['Models']
            if model=='LR':
                regressor=LinearRegression()
            elif model=='Lasso':
                regressor=Lasso(alpha=configDict['alphalas'])
            elif model=='Ridge':
                regressor=Ridge(alpha=configDict['alpharid'])
            elif model=='RF':
                regressor=RandomForestRegressor(n_estimators=configDict['n_estimatorsrf'],random_state=42)
            elif model=='XGboost':
                regressor=XGBRegressor(n_estimators=configDict['n_estimatorsxg'],random_state=42)
            regressor.fit(self.X_train,self.Y_train)
            y_pred = regressor.predict(self.X_test)
            mse = mean_squared_error(self.Y_test, y_pred)
            print("Mean Squared Error:", mse)
            return {'loss':mse,'time':time.time()-start_time}
    def timeser(self,configDict):
        model=configDict['Models']
        if model=='Arima':
            return Arima(self.Y_train,self.Y_test,p=configDict['p'],d=configDict['d'],q=configDict['q'],freq=self.freq)
        elif model=='Sarima':
            return 10000
            
            


In [83]:
from enum import Enum

class ProblemType(Enum):
    CLASSIFICATION = 'classification'
    REGRESSION = 'regression'
    TIME_SERIES = 'time series'
    UNBALANCED = 'unbalanced'

class Facade:
    def __init__(self, problem_type,Models,X_train,X_test,Y_train,Y_test,freq="d",m=7):
        if isinstance(problem_type, ProblemType):
            self.problem_type = problem_type
            self.models=Models
            self.X_train=X_train
            self.X_test=X_test
            self.Y_train=Y_train
            self.Y_test=Y_test
            self.freq=freq
            self.m=m
        else:
            raise ValueError("problem_type must be a ProblemType Enum")
    def chooseFacade(self):
        if self.problem_type==ProblemType.CLASSIFICATION:
            return  self.ClassificationFacade()
        elif self.problem_type==ProblemType.REGRESSION:
            return self.RegressionFacade()
        elif self.problem_type==ProblemType.TIME_SERIES:
            return self.TimeSeriesFacade()
        elif self.problem_type==ProblemType.UNBALANCED:
            return self.UnbalancedFacade()
    def ClassificationFacade(self):
        classifier=Models(self.models,'Classification',self.X_train,self.Y_train,self.X_test,self.Y_test)
        scenario = Scenario(classifier.configspace(), deterministic=True,objectives=['loss','time'], n_trials=100)
        smac = HyperparameterOptimizationFacade(scenario, classifier.train,overwrite=True,callbacks=[CustomCallback()],
                                                multi_objective_algorithm=HyperparameterOptimizationFacade.get_multi_objective_algorithm(scenario,objective_weights=[2, 1]))
        incumbents = smac.optimize()
        for incumbent in incumbents:
            print(incumbent)
        return incumbents
    def RegressionFacade(self):
        Regressor=Models(self.models,'Regression',self.X_train,self.Y_train,self.X_test,self.Y_test)
        scenario = Scenario(Regressor.configspace(), deterministic=True,objectives=['loss','time'], n_trials=100)
        smac = HyperparameterOptimizationFacade(scenario, Regressor.train,overwrite=True,callbacks=[CustomCallback()],
                                                multi_objective_algorithm=HyperparameterOptimizationFacade.get_multi_objective_algorithm(scenario,objective_weights=[2, 1]))
        incumbents = smac.optimize()
        for incumbent in incumbents:
            print(incumbent)
        return incumbents
    def TimeSeriesFacade(self):
        timeclassifier=Models(self.models,'TimeSeries',self.X_train,self.Y_train,self.X_test,self.Y_test,freq=self.freq,m=self.m)
        scenario = Scenario(timeclassifier.configspace(), deterministic=True, n_trials=100)
        smac = HyperparameterOptimizationFacade(scenario, timeclassifier.train,overwrite=True,callbacks=[CustomCallback()])
        incumbent=smac.optimize()
        print(incumbent)
        return incumbent
        
    def UnbalancedFacade(self):
        pass

In [68]:

target = "class_good"
X = df.drop(columns=target)
y = df[target]

# Split the dataset into training and testing sets
X_train_raw, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X =X[selected_features]
X_train_raw = X_train_raw[selected_features]
X_test = X_test[selected_features]
print(X_test.shape)
if __name__ =="__main__":
#test classifcation
    classincummbet=Facade(ProblemType.CLASSIFICATION,['KNN','LR','RF'],X_train_raw,X_test,y_train,y_test)
    classincummbet.chooseFacade()
#     print(classincummbet)

    


(200, 15)
[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 40 to 25 (max_ratio == 0.25).
[INFO][abstract_initial_design.py:147] Using 24 initial design configurations and 0 additional configurations.
let's start the optimization
[INFO][abstract_intensifier.py:305] Using only one seed for deterministic scenario.
config_dict:{'Models': 'KNN', 'Ks': 7}
the type of the classifier is: <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
the loss is:  0.32999999999999996
[INFO][abstract_intensifier.py:515] Added config 61f4fc as new incumbent because there are no incumbents yet.
The number of trials is: 1
config_dict:{'Models': 'RF', 'n_estimators': 13}
the type of the classifier is: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
the loss is:  0.265
[INFO][abstract_intensifier.py:602] Config 7d1035 is a new incumbent. Total number of incumbents: 2.
The number of trials is: 2
config_dict:{'Models': 'LR', 'regularizationStre': 0.49

In [69]:
path = r"C:\Users\Alaa\Desktop\archive\daily-minimum-temperatures-in-me.csv"
df =pd.read_csv(path)
df.rename(columns={'Date': 'ds', 'Daily minimum temperatures': 'y'}, inplace=True)
df['ds'] = pd.to_datetime(df['ds'], format='%m/%d/%Y')
df['y'] = df['y'].str.replace('[^0-9\.]', '', regex=True)
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df['y'] = df['y'].astype(float)
df.set_index('ds', inplace=True)
split_date = pd.to_datetime('1990-12-15')
train_data = df[df.index <= split_date]
test_data = df[df.index > split_date]
traindatax='lol'
traindatay=train_data['y']
test_datay=test_data['y']
test_datax='loll'


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Alaa\\Desktop\\archive\\daily-minimum-temperatures-in-me.csv'

In [ ]:
train_data

,y
ds,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8
...,...
1990-12-11,11.1
1990-12-12,14.0
1990-12-13,11.4


In [ ]:
pd.DataFrame(traindatay)

,y
ds,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8
...,...
1990-12-11,11.1
1990-12-12,14.0
1990-12-13,11.4


In [ ]:
test_datay

ds
1990-12-16    13.6
1990-12-17    13.9
1990-12-18    17.2
1990-12-19    14.7
1990-12-20    15.4
1990-12-21    13.1
1990-12-22    13.2
1990-12-23    13.9
1990-12-24    10.0
1990-12-25    12.9
1990-12-26    14.6
1990-12-27    14.0
1990-12-28    13.6
1990-12-29    13.5
1990-12-30    15.7
1990-12-31    13.0
Name: y, dtype: float64

In [ ]:
print(Arima(pd.DataFrame(traindatay),pd.DataFrame(test_datay),p=1,d=1,q=2,freq='d'))

7.63395894253463


In [ ]:
# if __name__=="__main__":
    
#     classincummbet=Facade(ProblemType.TIME_SERIES,['Arima','Sarima'],traindatax,test_datax,pd.DataFrame(traindatay),pd.DataFrame(test_datay))
#     incumbent=classincummbet.chooseFacade()

ValueError: Hyperparameter 'regularizationStre' is conditional on the illegal value 'SVC' of its parent hyperparameter 'Models'

In [77]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data[:, 1:]
y = iris.data[:, 0] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.10212647866320382


In [81]:
if __name__=="__main__":
    regreincumbent=Facade(ProblemType.REGRESSION,['LR','Lasso','Ridge','RF'],X_train,X_test,y_train,y_test)
    regreincumbent.chooseFacade()

[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 40 to 25 (max_ratio == 0.25).
[INFO][abstract_initial_design.py:147] Using 20 initial design configurations and 0 additional configurations.
let's start the optimization
[INFO][abstract_intensifier.py:305] Using only one seed for deterministic scenario.
config_dict:{'Models': 'Lasso', 'alphalas': 62.060263897813854}
Mean Squared Error: 0.719450694444445
[INFO][abstract_intensifier.py:515] Added config 6bec55 as new incumbent because there are no incumbents yet.
The number of trials is: 1
config_dict:{'Models': 'RF', 'n_estimatorsrf': 12}
Mean Squared Error: 0.0982300668724279
[INFO][abstract_intensifier.py:602] Config f7901f is a new incumbent. Total number of incumbents: 2.
The number of trials is: 2
config_dict:{'Models': 'Ridge', 'alpharid': 27.794627256914975}
Mean Squared Error: 0.10962616257680025
[INFO][abstract_intensifier.py:594] Added config 0e0f3b and rejected config 6bec55 as incumbent 

In [ ]:
regressor=XGBRegressor(n_estimators=10,random_state=42)
regressor.fit(X_train,y_train)
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
regressor=RandomForestRegressor(n_estimators=19)
regressor.fit(X_train,y_train)
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
Classifier=RandomForestClassifier(n_estimators=19)
Classifier.fit(X_train_raw,y_train)
y_pred = Classifier.predict(X_test)
loss=1-accuracy_score(y_test,y_pred)
print("the loss is: ",loss)